In [1]:
from pysam import FastaFile
import numpy as np
import random
from generate_downstream_data import Downstream
import pandas as pd
from pickled_downstream_data import SeqDataset

In [2]:
#-15000 downstream, 250000 upstream
TATA_filename = 'TATA_Promoter.fa'
TATA_sequences = FastaFile(TATA_filename)

noTATA_filename = 'noTATA_Promoter.fa'
noTATA_sequences = FastaFile(noTATA_filename)

TSS_Site = 15001

In [3]:
pos_start, pos_end = TSS_Site-7500, TSS_Site-7500+100000
TATA_positive_class, noTATA_positive_class = list(), list()

for r1 in TATA_sequences.references:
    TATA_positive_class.append(TATA_sequences.fetch(r1, pos_start, pos_end))
    
for r2 in noTATA_sequences.references:
    noTATA_positive_class.append(noTATA_sequences.fetch(r2, pos_start, pos_end))

positive_class = TATA_positive_class + noTATA_positive_class

In [4]:
len(positive_class)

29598

In [5]:
TATA_negative_class = list()

for r1 in TATA_sequences.references:
    neg_start = TSS_Site+250
    flag = True
    while flag:
        seq = TATA_sequences.fetch(r1, neg_start, neg_start+100000)
        if 'TATA' in seq:
            TATA_negative_class.append(seq)
            flag = False
        else:
            neg_start += 100

In [6]:
def generate_negative(sequence):
    seq_list = []
    for i in range(0, len(sequence), int(len(sequence)/20)):
        seq_list.append(sequence[i:i + int(len(sequence)/20)])
        
    indices = random.sample([i for i in range(len(seq_list))], 12)
    
    for idx in indices:
        random_string = ''
        for _ in range(len(seq_list[idx])):
            random_string += random.choice(['A', 'G', 'C', 'T'])
        seq_list[idx] = random_string
        
    negative = ''.join(seq_list)
    return negative
generate_negative(noTATA_positive_class[0])

'...'

In [7]:
c = Downstream('TATA_Promoter.fa', 'noTATA_Promoter.fa')

In [8]:
c.get_positive_class()[0]

('...',
 1)

In [9]:
c.get_negative_class()[0]

('...',
 0)

In [10]:
data = c.get_positive_class() + c.get_negative_class()
df = pd.DataFrame.from_records(data, columns=['DNA_String', 'Is_Promoter_Sequence'])
df

,DNA_String,Is_Promoter_Sequence
0,TTATGCTATGGACAGTAGGTGCCAGGCTCTGTGTCAGCAACTCTGC...,1
1,GCTTTGTAGGTCATTGCTCTAAATTTCTCACCCTGCTGTTTCTTAA...,1
2,CTCAGCTTACTGCTACCTCCACCTCCCAGGTTCAAGCGATTCTCCC...,1
3,AAAGGACCAACAGGATATGACAATGAAATGCAGTGGATGATTCTGG...,1
4,TACATAAGGAGCTGAGAATCCAAGCCAACACTGGAGGCAACACACA...,1
...,...,...
59191,TCCCTAATAAATGTCTCTCCCATAGTGTCCAGTGAAAAGTTTACTA...,0
59192,CATAAGGACACGAATCTATCATTCTCTGTGCAGATCCCCCAAAATT...,0
59193,TGCGGGTATGTTTGGCCCAGACGGTAGTGACGGGAACCCCTGGGAC...,0
59194,TATATAGCTATATAATCTATAATTATAGATTATAGCACTTTGGGAG...,0


In [11]:
df = df.sample(frac=1).reset_index(drop=True)
df.to_pickle('Downstream_Data.pkl')

In [12]:
df = pd.read_pickle('Downstream_Data.pkl')
df

,DNA_String,Is_Promoter_Sequence
0,CCGGCCCACATGTTCAGAGCGGTCGACTCACGAGATTACGTAGCAC...,0
1,GCTAGTTAATTTGATCATATAAATCCTAAATAGGCCCAGGCTCGGT...,0
2,GGAGCTCATCTGCACCTCTCGGCATCCCTTAGGATGGAGCCGACCT...,0
3,TTTAAGCTGAGCTAAATAAAGCTAATATTTGTGCTACTAAAGATGT...,1
4,ACCCAATAGGGTGTGTTGGAGAAAGGCCTTTTGAAACTTGGGGTTC...,0
...,...,...
59191,AGGTGGGAAGGCTCCAGATGTGCCAAGTGGGGAGTCCAAAGTGGGG...,0
59192,CACCATCACCACCCGCACCCTGCGCACCCCATGCTCCACGAACCCT...,0
59193,ACAATAATTTACAACCACAAGAATGGAAGAGTCAATTCTGAGAAGA...,1
59194,GTGTGGCTGGGGTCTGGGCTCTGGGGTCTGAGGGAGGAGGGGCTGG...,0


In [13]:
data = SeqDataset('Downstream_Data.pkl', 1000)

In [14]:
print(data.__getitem__())

(1, 100000)
(tensor([[1., 1., 2.,  ..., 0., 1., 0.]], dtype=torch.float64), 0)


In [15]:
#A = 